# Environment variables

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import conf

In [ ]:
N_JOBS = conf.GENERAL["N_JOBS"]
display(N_JOBS)

In [ ]:
%env MKL_NUM_THREADS=$N_JOBS
%env OPEN_BLAS_NUM_THREADS=$N_JOBS
%env NUMEXPR_NUM_THREADS=$N_JOBS
%env OMP_NUM_THREADS=$N_JOBS

# Modules

In [ ]:
from pathlib import Path

import pandas as pd
from tqdm import tqdm

from gls import GLSPhenoplier

# Settings

In [ ]:
OUTPUT_DIR = conf.RESULTS["GLS"]
display(OUTPUT_DIR)

OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# Load data

## PhenomeXcan (S-MultiXcan)

In [ ]:
# INPUT_SUBSET = "z_score_std"

In [ ]:
# INPUT_STEM = "projection-smultixcan-efo_partial-mashr-zscores"

In [ ]:
# input_filepath = Path(
#     conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
#     INPUT_SUBSET,
#     f"{INPUT_SUBSET}-{INPUT_STEM}.pkl",
# ).resolve()

In [ ]:
# data = pd.read_pickle(input_filepath)

In [ ]:
# data.shape

In [ ]:
# data.head()

## MultiPLIER summary

In [ ]:
multiplier_model_summary = pd.read_pickle(conf.MULTIPLIER["MODEL_SUMMARY_FILE"])

In [ ]:
multiplier_model_summary.shape

In [ ]:
multiplier_model_summary.head()

In [ ]:
well_aligned_lvs = multiplier_model_summary[
    (multiplier_model_summary["FDR"] < 0.05) | (multiplier_model_summary["AUC"] >= 0.75)
]

display(well_aligned_lvs.shape)
display(well_aligned_lvs.head())

In [ ]:
well_aligned_lv_codes = set([f"LV{lvi}" for lvi in well_aligned_lvs["LV index"]])

In [ ]:
len(well_aligned_lv_codes)

In [ ]:
list(well_aligned_lv_codes)[:5]

## eMERGE traits info

In [ ]:
# FIXME: hardcoded
input_filepath = Path(
    "/home/miltondp/projects/labs/greenelab/phenoplier/base/data/emerge",
    "eMERGE_III_PMBB_GSA_v2_2020_phecode_AFR_EUR_cc50_counts_w_dictionary.txt",
).resolve()
display(input_filepath)

In [ ]:
emerge_traits_df = pd.read_csv(
    input_filepath,
    sep="\t",
    dtype={"phecode": str},
    usecols=["phecode", "phenotype", "category"],
)

In [ ]:
emerge_traits_df = emerge_traits_df.rename(
    columns={
        "phenotype": "phecode_phenotype",
        "category": "phecode_category",
    }
)

In [ ]:
emerge_traits_df.shape

In [ ]:
emerge_traits_df.head()

## eMERGE (S-MultiXcan)

In [ ]:
# FIXME: path hardcoded
emerge_smultixcan_zscores_filepath = Path(
    "/home/miltondp/projects/labs/greenelab/phenoplier/base/data/emerge/gene_assoc/emerge-smultixcan-mashr-zscores.pkl"
).resolve()

display(emerge_smultixcan_zscores_filepath)

In [ ]:
_tmp = pd.read_pickle(emerge_smultixcan_zscores_filepath)

In [ ]:
_tmp.shape

In [ ]:
_tmp.head()

## GLS results on PhenomeXcan

In [ ]:
input_filepath = conf.RESULTS["GLS"] / "gls_phenotypes-crispr_lvs.pkl"
display(input_filepath)

In [ ]:
gls_phenomexcan_crispr = pd.read_pickle(input_filepath)

In [ ]:
gls_phenomexcan_crispr.shape

In [ ]:
gls_phenomexcan_crispr.head()

## GLS results on eMERGE

In [ ]:
# input_filepath = conf.RESULTS["GLS"] / "gls_phenotypes-emerge.pkl"
# display(input_filepath)

In [ ]:
# gls_emerge = pd.read_pickle(input_filepath)

In [ ]:
# gls_emerge.shape

In [ ]:
# gls_emerge.head()

# Select LV from previous GLS run on PhenomeXcan

In [ ]:
gls_phenomexcan_lvs = (
    gls_phenomexcan_crispr[["lv", "lv_set"]].drop_duplicates().reset_index(drop=True)
)

In [ ]:
gls_phenomexcan_lvs.shape

In [ ]:
gls_phenomexcan_lvs.head()

# Select traits from previous GLS run on eMERGE

In [ ]:
emerge_traits_df["phecode_category"].unique()

In [ ]:
gls_traits = emerge_traits_df[
    emerge_traits_df["phecode_category"].isin(
        [
            #     "hematopoietic",
            "circulatory system",
            "endocrine/metabolic",
        ]
    )
]["phecode"].unique()

In [ ]:
gls_traits.shape

In [ ]:
gls_traits

# GLSPhenoplier

## Get list of phenotypes/lvs pairs

In [ ]:
phenotypes_lvs_pairs = []

for idx, lv_row in gls_phenomexcan_lvs.iterrows():
    for phenotype_code in gls_traits:
        phenotypes_lvs_pairs.append(
            {
                "phenotype": phenotype_code,
                "lv": lv_row["lv"],
                "lv_set": lv_row["lv_set"],
            }
        )

phenotypes_lvs_pairs = pd.DataFrame(phenotypes_lvs_pairs).drop_duplicates()

In [ ]:
phenotypes_lvs_pairs = phenotypes_lvs_pairs.sort_values("phenotype").reset_index(
    drop=True
)

In [ ]:
phenotypes_lvs_pairs.shape

In [ ]:
phenotypes_lvs_pairs.head()

## Run

In [ ]:
output_file = OUTPUT_DIR / "gls_phenotypes-emerge-crispr_lvs.pkl"
display(output_file)

In [ ]:
results = []

pbar = tqdm(total=phenotypes_lvs_pairs.shape[0])

for idx, row in phenotypes_lvs_pairs.iterrows():
    phenotype_code = row["phenotype"]
    lv_code = row["lv"]

    pbar.set_description(f"{phenotype_code} - {lv_code}")

    gls_model = GLSPhenoplier(
        smultixcan_result_set_filepath=emerge_smultixcan_zscores_filepath
    )
    gls_model.fit_named(lv_code, phenotype_code)
    res = gls_model.results

    results.append(
        {
            "phenotype": phenotype_code,
            "lv": lv_code,
            "lv_set": row["lv_set"],
            "lv_with_pathway": lv_code in well_aligned_lv_codes,
            "coef": res.params.loc["lv"],
            "pvalue": res.pvalues.loc["lv"],
            "summary": gls_model.results_summary,
        }
    )

    if (idx % 10) == 0:
        pd.DataFrame(results).to_pickle(output_file)

    pbar.update(1)

pbar.close()

In [ ]:
results = pd.DataFrame(results)

In [ ]:
results.shape

In [ ]:
results.head()

In [ ]:
results.sort_values("pvalue").head(10)

## Save

In [ ]:
results.to_pickle(output_file)